In [1]:
%load_ext autoreload
%autoreload 2

import os
from dotenv import load_dotenv
load_dotenv()

OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

In [2]:
prompt_text = "Given this product title, please select between 3 and 6 criteria to rate in order to compose a product review. No need to explain the criteria."
input_product = "Nike Men's Revolution 5 Running Shoes"

### Chunking

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema.document import Document

def get_text_chunks(text) -> list[Document]:
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    docs = [Document(page_content=x) for x in text_splitter.split_text(text)]
    return docs

In [4]:
texts = get_text_chunks(input_product)
texts

[Document(page_content="Nike Men's Revolution 5 Running Shoes")]

### QA Retrieval Chain

In [5]:
from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.chains.retrieval_qa.base import BaseRetrievalQA
from langchain.chains import RetrievalQA


OPENAI_MODEL = os.environ.get('OPENAI_MODEL') # 'gpt-3.5-turbo'
BASE_PERSIST_DIRECTORY = '../db/chroma_3/'


def embed_texts(texts, model_name) -> list[Document]:
    if model_name == 'openai':
        embedder = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
    else:
        embedder = HuggingFaceEmbeddings(model_name=model_name)

    vector_db = Chroma.from_documents(
        texts,
        embedder,
        persist_directory=BASE_PERSIST_DIRECTORY + model_name
    )
    return vector_db

def get_qa(model, vector_db) -> BaseRetrievalQA:
    qa = RetrievalQA.from_chain_type(
        llm=model,
        chain_type="stuff",
        retriever=vector_db.as_retriever(search_kwargs={"k": 1}),
    return_source_documents=False,
    verbose=False,
    )
    return qa

### OpenAI

In [6]:
from langchain_openai import ChatOpenAI

llm_openai = ChatOpenAI(model_name=OPENAI_MODEL, openai_api_key=OPENAI_API_KEY)
print(llm_openai, '\n')

vector_db = embed_texts(texts, model_name='openai')
qa_openai = get_qa(llm_openai, vector_db)
print(qa_openai, '\n')

answer = qa_openai.invoke(prompt_text)
print(answer["result"])

client=<openai.resources.chat.completions.Completions object at 0x7f168b4bb7c0> async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7f168b4ecf10> openai_api_key=SecretStr('**********') openai_proxy='' 

combine_documents_chain=StuffDocumentsChain(llm_chain=LLMChain(prompt=ChatPromptTemplate(input_variables=['context', 'question'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template="Use the following pieces of context to answer the user's question. \nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n----------------\n{context}")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='{question}'))]), llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7f168b4bb7c0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7f168b4ecf10>, openai_api_key=SecretStr('**********'), openai_proxy=''

### Local

In [7]:
HG_EMBEDDING_MODEL = 'sentence-transformers/all-MiniLM-L6-v2'
#MODEL_NAME = 'mistral-7b-openorca.gguf2.Q4_0.gguf'  # GPT4All model
MODEL_NAME = 'orca-mini-3b-gguf2-q4_0.gguf'
MODEL_PATH = '../models/' + MODEL_NAME
!curl -C - -o {MODEL_PATH} https://gpt4all.io/models/gguf/{MODEL_NAME}

** Resuming transfer from byte position 1979946720
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 27242    0 27242    0     0  58433      0 --:--:-- --:--:-- --:--:-- 58334


In [8]:
from langchain.callbacks.streaming_stdout_final_only import FinalStreamingStdOutCallbackHandler
from langchain.llms import GPT4All

callbacks = [FinalStreamingStdOutCallbackHandler()]
llm_local =  GPT4All(model=MODEL_PATH, callbacks=callbacks, verbose=True)
print('llm_local----->  ', llm_local, '\n')

vector_db =  embed_texts(texts, model_name=HG_EMBEDDING_MODEL)
qa_local = get_qa(llm_local, vector_db)
print('qa_local----->  ', qa_local, '\n')

answer =  qa_local.invoke(prompt_text)
print('answer----->  ',answer["result"])

Failed to load libllamamodel-mainline-cuda.so: dlopen: libcuda.so.1: cannot open shared object file: No such file or directory
Failed to load libllamamodel-mainline-cuda-avxonly.so: dlopen: libcuda.so.1: cannot open shared object file: No such file or directory


llm_local----->   GPT4All
Params: {'model': '../models/orca-mini-3b-gguf2-q4_0.gguf', 'max_tokens': 200, 'n_predict': 256, 'top_k': 40, 'top_p': 0.1, 'temp': 0.7, 'n_batch': 8, 'repeat_penalty': 1.18, 'repeat_last_n': 64, 'streaming': False} 



/home/shahab/.pyenv/versions/3.10.6/envs/review_env/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/home/shahab/.pyenv/versions/3.10.6/envs/review_env/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/home/shahab/.pyenv/versions/3.10.6/envs/review_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be remo

qa_local----->   combine_documents_chain=StuffDocumentsChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['context', 'question'], template="Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\n{context}\n\nQuestion: {question}\nHelpful Answer:"), llm=GPT4All(verbose=True, callbacks=[<langchain.callbacks.streaming_stdout_final_only.FinalStreamingStdOutCallbackHandler object at 0x7f169b0ffc10>], model='../models/orca-mini-3b-gguf2-q4_0.gguf', client=<gpt4all.gpt4all.GPT4All object at 0x7f169b0ff340>)), document_variable_name='context') retriever=VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7f151c6b3d60>, search_kwargs={'k': 1}) 

answer----->    4 criteria (e.g., comfort, durability, style, performance)


### LLM Chain

In [28]:
from langchain.prompts import PromptTemplate

prompt_text2 = "Please list between 3 and 6 criteria that could be rated for this product. Just return the list of criteria, nothing more."
#template = f"""Product: '{{product_type}}'\n{prompt_text2}""".format(product_type=input_product)
#prompt = PromptTemplate(template=template, input_variables=["product_type"])



template = f"""Product: '{{product_type}}'
{prompt_text2}"""
prompt = PromptTemplate(template=template, input_variables=["product_type"])

print('prompt---> ', prompt)


prompt--->  input_variables=['product_type'] template="Product: '{product_type}'\nPlease list between 3 and 6 criteria that could be rated for this product. Just return the list of criteria, nothing more."


In [29]:
from langchain.chains import LLMChain

llm_chain = LLMChain(prompt=prompt, llm=llm_local, return_final_only=True)
llm_chain
print('llm_chain---> ', llm_chain, '\n')


llm_chain--->  prompt=PromptTemplate(input_variables=['product_type'], template="Product: '{product_type}'\nPlease list between 3 and 6 criteria that could be rated for this product. Just return the list of criteria, nothing more.") llm=GPT4All(verbose=True, callbacks=[<langchain.callbacks.streaming_stdout_final_only.FinalStreamingStdOutCallbackHandler object at 0x7f169b0ffc10>], model='../models/orca-mini-3b-gguf2-q4_0.gguf', client=<gpt4all.gpt4all.GPT4All object at 0x7f169b0ff340>) 



In [30]:
res =  llm_chain.invoke(input_product)
res

{'product_type': "Nike Men's Revolution 5 Running Shoes", 'text': ''}

In [33]:
from langchain_openai import OpenAI

llm = OpenAI(openai_api_key=OPENAI_API_KEY)
llm_chain = LLMChain(prompt=prompt, llm=llm, return_final_only=True)
res_openai = llm_chain.invoke(input_product)
result = res_openai['text']
result

'\n\n1. Comfort level\n2. Durability\n3. Breathability\n4. Cushioning\n5. Traction\n6. Design/style'